In [6]:
# 安装必要的依赖
!pip install yfinance seaborn scipy matplotlib pandas --quiet



In [7]:
!pip install streamlit yfinance matplotlib scipy pyngrok


In [8]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.stats import norm
from scipy.optimize import fsolve
import ipywidgets as widgets
from IPython.display import display

# Interactive widgets
stock_input = widgets.Text(value='AAPL', description='Ticker:')
expiry_slider = widgets.IntSlider(value=3, min=1, max=10, step=1, description='Expiry Count:')
E_slider = widgets.FloatSlider(value=0.01, min=0.001, max=0.05, step=0.001, description='E (Pinning):', readout_format='.3f')
option_type = widgets.Dropdown(options=['Put', 'Call'], value='Put', description='Option Type:')

# Display widgets
display(stock_input, expiry_slider, E_slider, option_type)

# ================= Black-Scholes Option Pricing =================
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    if T <= 0 or sigma <= 0:
        raise ValueError("Time to maturity and volatility must be positive.")
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type.lower() == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type.lower() == 'put':
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option_type. Use 'call' or 'put'.")

# ================= Earnings Expected Move =================
def earnings_expected_move(ticker):
    stock = yf.Ticker(ticker)
    latest_price = stock.history(period="1d")['Close'][-1]
    expiry_dates = stock.options
    option_chain = stock.option_chain(expiry_dates[0])
    calls, puts = option_chain.calls, option_chain.puts
    atm_strike = min(calls['strike'], key=lambda x: abs(x - latest_price))
    atm_call = calls[calls['strike'] == atm_strike]['lastPrice'].values[0]
    atm_put = puts[puts['strike'] == atm_strike]['lastPrice'].values[0]
    expected_move = (atm_call + atm_put) / latest_price
    print(f"[Earnings Expected Move] {ticker}: ±{expected_move*100:.2f}%")
    return expected_move

# ================= Volatility Skew & Risk Reversal =================
def vol_skew_and_rr(ticker):
    stock = yf.Ticker(ticker)
    expiry_dates = stock.options
    option_chain = stock.option_chain(expiry_dates[0])
    calls, puts = option_chain.calls, option_chain.puts
    latest_price = stock.history(period="1d")['Close'][-1]
    otm_call_strike = min(calls['strike'], key=lambda x: abs(x - latest_price*1.1))
    otm_put_strike = min(puts['strike'], key=lambda x: abs(x - latest_price*0.9))
    iv_call = calls[calls['strike'] == otm_call_strike]['impliedVolatility'].values[0]
    iv_put = puts[puts['strike'] == otm_put_strike]['impliedVolatility'].values[0]
    risk_reversal = iv_call - iv_put
    print(f"[Vol Skew & RR] Call IV: {iv_call:.2%}, Put IV: {iv_put:.2%}, RR: {risk_reversal:.2%}")
    return iv_call, iv_put, risk_reversal

# ================= Max Pain Calculation =================
def calculate_max_pain(ticker, expiry):
    stock = yf.Ticker(ticker)
    option_chain = stock.option_chain(expiry)
    calls, puts = option_chain.calls, option_chain.puts

    strikes = sorted(set(calls['strike']).union(set(puts['strike'])))
    pain = []
    for K in strikes:
        call_loss = np.sum(np.maximum(0, K - calls['strike']) * calls['openInterest'])
        put_loss = np.sum(np.maximum(0, puts['strike'] - K) * puts['openInterest'])
        pain.append(call_loss + put_loss)
    min_pain_idx = np.argmin(pain)
    max_pain_price = strikes[min_pain_idx]
    print(f"[Max Pain] {ticker} ({expiry}) Max Pain Price: {max_pain_price}")

    plt.figure(figsize=(10, 5))
    plt.plot(strikes, pain, marker='o')
    plt.axvline(x=max_pain_price, color='r', linestyle='--', label=f"Max Pain Price ({max_pain_price})")
    plt.title(f"Max Pain Theory: {ticker} ({expiry})")
    plt.xlabel("Strike Price")
    plt.ylabel("Total Option Holder Loss (Pain)")
    plt.legend()
    plt.grid(True)
    plt.show()
    return max_pain_price

# ================= Put-Call Parity & BS Implied Price =================
def implied_price_analysis(ticker, expiry):
    aapl = yf.Ticker(ticker)
    dividend_yield = aapl.dividends[-4:].sum() / aapl.history(period="1d")['Close'].iloc[-1] if len(aapl.dividends) > 0 else 0
    option_chain = aapl.option_chain(expiry)
    calls, puts = option_chain.calls, option_chain.puts
    S_market = aapl.history(period="1d")['Close'].iloc[-1]
    atm_call = calls.iloc[(calls['strike'] - S_market).abs().argmin()]
    atm_put = puts.iloc[(puts['strike'] - S_market).abs().argmin()]
    K = atm_call['strike']
    expiry_date = datetime.strptime(expiry, "%Y-%m-%d")
    T = (expiry_date - datetime.now()).days / 365
    t_bill = yf.Ticker("^IRX")
    r = t_bill.history(period="1d")['Close'].iloc[-1] / 100 if not t_bill.history(period="1d").empty else 0.02

    C = atm_call['lastPrice']
    P = atm_put['lastPrice']
    S_parity = C - P + K * np.exp(-(r - dividend_yield) * T)

    sigma = atm_call['impliedVolatility']
    def objective(S):
        d1 = (np.log(S / K) + (r - dividend_yield + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return S * np.exp(-dividend_yield*T) * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2) - C
    S_bs = fsolve(objective, S_market)[0]

    results = {
        "Market Price": round(S_market, 2),
        "Dividend Yield": f"{dividend_yield*100:.2f}%",
        "Risk-Free Rate": f"{r*100:.2f}%",
        "Put-Call Implied Price": round(S_parity, 2),
        "BS Model Implied Price": round(S_bs, 2),
        "Strike Price Used": K,
        "Expiration Date": expiry,
        "Call Option Price": C,
        "Put Option Price": P,
        "Implied Volatility (IV)": f"{sigma*100:.2f}%"
    }
    df_results = pd.DataFrame(list(results.items()), columns=["Metric", "Value"])
    print("\n[Implied Price Analysis]")
    display(df_results)

    return S_parity, S_bs

# ================= Combined Interactive Function =================
def calculate_and_plot(ticker, expiry_count, E, opt_type):
    ticker = ticker.upper()
    stock = yf.Ticker(ticker)
    expirations = stock.options

    # Fetch historical stock data
    end_date = datetime.today()
    start_date = end_date - timedelta(days=365)
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    print("\nRecent stock data:")
    display(stock_data.tail())

    print("Available option expiry dates:", expirations)
    expiry = expirations[0]
    option_chain = stock.option_chain(expiry)
    calls, puts = option_chain.calls, option_chain.puts

    max_oi_call = calls.loc[calls['openInterest'].idxmax()]
    max_oi_put = puts.loc[puts['openInterest'].idxmax()]
    print("Call Max OI Strike:", max_oi_call['strike'])
    print("Put Max OI Strike:", max_oi_put['strike'])

    recent_data = stock_data.tail(10)
    plt.figure(figsize=(10,6))
    plt.plot(recent_data.index, recent_data['Close'], label=f"{ticker} Close Price", linewidth=2)
    plt.axhline(y=max_oi_call['strike'], color='r', linestyle='--', label=f"Call Max OI ({max_oi_call['strike']})")
    plt.axhline(y=max_oi_put['strike'], color='g', linestyle='--', label=f"Put Max OI ({max_oi_put['strike']})")
    plt.title(f"{ticker} Price vs Option Max OI Strikes (Near Expiration)")
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Pinning probability calculation
    results = []
    for expiry in expirations[:expiry_count]:
        expiry_date = datetime.strptime(expiry, "%Y-%m-%d")
        option_chain = stock.option_chain(expiry)
        options_df = option_chain.puts if opt_type == 'Put' else option_chain.calls
        max_oi = options_df.loc[options_df['openInterest'].idxmax()].squeeze()
        K = float(max_oi['strike'])
        OI = float(max_oi['openInterest'])
        IV = float(max_oi['impliedVolatility'])
        price_window = stock_data.loc[:expiry_date].tail(5)
        if price_window.empty:
            continue
        S0 = float(price_window['Close'].iloc[-1])
        V = float(price_window['Volume'].mean())
        today = price_window.index[-1].to_pydatetime()
        days_to_expiry = max((expiry_date - today).days, 1)
        T = days_to_expiry / 252
        zeta0 = np.log(S0 / K) / (IV * np.sqrt(T))
        beta = (E * OI) / (IV * np.sqrt(T) * V)
        P_pinning = 1 - np.exp(-2 * beta * np.exp(-zeta0**2 / 2))
        expiry_price_data = stock_data.loc[expiry_date:expiry_date]
        expiry_price = float(expiry_price_data['Close'].iloc[0]) if not expiry_price_data.empty else None
        deviation = abs(expiry_price - K) / K * 100 if expiry_price else None
        results.append({
            "expiry": expiry,
            "S0 (pre-expiry)": S0,
            "K (max OI)": K,
            "OI": OI,
            "IV": IV,
            "Volume": V,
            "Days_to_expiry": days_to_expiry,
            "P_pinning (%)": P_pinning * 100,
            "Expiry_price": expiry_price,
            "Deviation (%)": deviation
        })
        calculate_max_pain(ticker, expiry)
        implied_price_analysis(ticker, expiry)

    df_results = pd.DataFrame(results)
    print("\nPinning Probability and Strike Analysis:")
    display(df_results)

    plt.figure(figsize=(12,6))
    plt.plot(df_results['expiry'], df_results['P_pinning (%)'], marker='o', label='Pinning Probability (%)')
    plt.plot(df_results['expiry'], df_results['Deviation (%)'], marker='s', label='Price Deviation (%)')
    plt.xticks(rotation=45)
    plt.xlabel("Option Expiry Date")
    plt.ylabel("Percentage (%)")
    plt.title(f"{ticker}: Pinning Probability vs Deviation ({opt_type}, E={E})")
    plt.legend()
    plt.grid(True)
    plt.show()

    latest_price = stock.history(period="1d")['Close'][-1]
    atm_strike = float(latest_price)
    T = 30/252; r = 0.02; sigma = 0.25
    call_price = black_scholes_price(latest_price, atm_strike, T, r, sigma, 'call')
    put_price = black_scholes_price(latest_price, atm_strike, T, r, sigma, 'put')
    print(f"[Black-Scholes] Call: {call_price:.2f}, Put: {put_price:.2f}")

    earnings_expected_move(ticker)
    vol_skew_and_rr(ticker)

widgets.interactive_output(calculate_and_plot, {
    'ticker': stock_input,
    'expiry_count': expiry_slider,
    'E': E_slider,
    'opt_type': option_type
})


Text(value='AAPL', description='Ticker:')

IntSlider(value=3, description='Expiry Count:', max=10, min=1)

FloatSlider(value=0.01, description='E (Pinning):', max=0.05, min=0.001, readout_format='.3f', step=0.001)

Dropdown(description='Option Type:', options=('Put', 'Call'), value='Put')

Output()